Import Libraries

In [53]:
import json
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
import re
from TweetTokenizer_modified import TweetTokenizer
import unicodedata
import emoji
import html

Import the Data

In [54]:
# Import the data
training_data= json.load(open("C:\\Users\\Jerem\\Documents\\Spring 2023\\EXIST\\dirty_data\\training\\EXIST2023_training.json", encoding='utf-8'))
test_data = json.load(open("C:\\Users\\Jerem\\Documents\\Spring 2023\\EXIST\\dirty_data\\test\\EXIST2023_test_clean.json", encoding='utf-8'))
dev_data = json.load(open("C:\\Users\\Jerem\\Documents\\Spring 2023\\EXIST\\dirty_data\\dev\\EXIST2023_dev.json", encoding='utf-8'))

train_gold_hard = json.load(open("C:\\Users\\Jerem\\Documents\\Spring 2023\\EXIST\\evaluation\\golds\\EXIST2023_training_task1_gold_hard.json", encoding='utf-8'))
dev_gold_hard = json.load(open("C:\\Users\\Jerem\\Documents\\Spring 2023\\EXIST\\evaluation\\golds\\EXIST2023_dev_task1_gold_hard.json", encoding='utf-8'))

In [55]:
# Matts cleanTweet Function
# Remove duplicated word
#     Return cleaned tweet and count of word
def remove_duplicate(tweet, target):

    # Split input string separated by space
    tweet = tweet.split(" ")

    target_bool = False # Bool for whether the target exists in the tweet
    count = 0 # Count of targets
    clean_tweet = '' # Tweet with additional targets removed

    # Iterate through words in tweet
    for word in tweet:

        # Check if word is target
        #     If it is not, add word to clean_tweet
        if word == target:

            # Increment count if 2+ target words were present
            #     otherwise, add target to clean_tweet and set target_bool to True
            if target_bool:
                count += 1
            else:
                count = 1
                clean_tweet += ' ' + word
                target_bool = True

        else:
            clean_tweet += ' ' + word

    return clean_tweet.strip(), count

# Replace upside-down punctation marks
#     Return cleaned tweet
def replace_punct(tweet, upside_down_punct, punct):

    # Split input string separated by space
    tweet = tweet.split(" ")

    clean_tweet = '' # Tweet with additional targets removed

    stack = [] # Stack of punctuation marks

    # Iterate through words in tweet
    for word in tweet:

        # Check if word is target
        #     If it is not, add word to clean_tweet
        if word == upside_down_punct: stack.append(upside_down_punct)

        elif word == punct:

            if stack:
                stack.pop()
                clean_tweet += ' ' + word
            else: clean_tweet += ' ' + word

        else: clean_tweet += ' ' + word

    for elem in stack:
        clean_tweet += ' ' + punct

    return clean_tweet.strip()

def normalize(tweet):
    clean_tweet = ''
    for char in tweet:
        val = ord(char)
        if val >= 119938 and val <= 120067:
            val -= 119841
        clean_tweet += chr(val)
    return clean_tweet

def cleanTweet(tweet):

    #print(f'Original Tweet: {tweet}')

    clean_tweet = html.unescape(tweet) # Convert html characters to unicode
    clean_tweet = unicodedata.normalize('NFKC', clean_tweet) # Normalize font
    clean_tweet = normalize(clean_tweet) # Fix weird fonts

    clean_tweet = re.sub(r'•͈ᴗ•͈', emoji.emojize(':smiling_face_with_tear:'), clean_tweet) # Convert •͈ᴗ•͈ into an emoji

    clean_tweet = re.sub(r'https://[a-zA-Z0-9/.:]+', '', clean_tweet) # Remove links

    clean_tweet = re.sub(r'(@[a-zA-Z]+)@([a-zA-Z])', '\g<1> @\g<2>', clean_tweet) # Add space between usernames

    clean_tweet = re.sub(r'([^ @][a-zA-z])@([a-zA-Z])', '\g<1>ATIDENTIFICATIONTAG\g<2>', clean_tweet) # Turn @ into an identification tag if it is not a username

    clean_tweet = re.sub(r"([a-zA-Z ])[´`‘’]([a-zA-Z])", "\g<1>'\g<2>", clean_tweet) # Convert ´ and ` when surrounded by letters
    clean_tweet = re.sub(r'([0-9])°', '\g<1> degrees', clean_tweet) # Convert ° into the word 'degrees' when directly after a number
    clean_tweet = re.sub(r'([0-9])%', '\g<1> percent', clean_tweet) # Convert % into the word 'percent' when directly after a number
    clean_tweet = re.sub(r'([a-zA-Z])[*]+([a-z])', '\g<1>astrickidentificationtag\g<2>', clean_tweet) # Convert censoring astricks into identification tags

    clean_tweet = re.sub(r'[.´`^¨~°|─­,;‘’"“”«»()\[\]{}®\$£€*%↓ِ\u0301\u200D]', ' ', clean_tweet) # Replace special characters with a space

    clean_tweet = re.sub(r'\u00A9\uFE0F', 'c', clean_tweet) # DOUBLE-CHECK THIS Replacing copyrite symbol with a 'c'

    clean_tweet = ' '.join(TweetTokenizer(strip_handles = True, reduce_len = True, preserve_case = False).tokenize(clean_tweet)) # Tokenise tweet

    clean_tweet = re.sub(r' :($|\s)', '\g<1> ', clean_tweet) # Replace colons with a space when they aren't part of a time

    clean_tweet = re.sub(r' / ', ' ', clean_tweet) # Remove backslashes
    clean_tweet = re.sub(r'\w*\d\w*', ' ', clean_tweet) # Remove words with numbers

    clean_tweet = re.sub(r'<3', emoji.emojize(':red_heart:'), clean_tweet) # Convert <3 into an emoji
    clean_tweet = re.sub(r'\+', ' plus ', clean_tweet) # Convert + into the word plus
    clean_tweet = re.sub(r'\-', ' minus ', clean_tweet) # Convert - into the word minus

    clean_tweet = re.sub(r'atidentificationtag', '@', clean_tweet) # Convert @ symbols back
    clean_tweet = re.sub(r'astrickidentificationtag', '*', clean_tweet) # Convert * symbols back

    clean_tweet = re.sub(r'[<>]', '', clean_tweet) # Remove < and >
    clean_tweet = re.sub(r' -($|\s)', ' ', clean_tweet) # Remove hyphens when not connecting words or numbers

    hashtag_regex = re.compile('#[\w]+')
    hashtag_lst = hashtag_regex.findall(clean_tweet)

    clean_tweet = re.sub(r'#[\w]+', '<HASHTAG>', clean_tweet) # Convert hashtags to <HASHTAG>
    clean_tweet = re.sub(r'usernameidentificationtag', '<USERNAME>', clean_tweet) # Convert usernames to <USERNAME>

    clean_tweet = re.sub(r"([a-z>]) '[\s]*s ", "\g<1>'s ", clean_tweet) # Reattach possesives

    clean_tweet, username_count = remove_duplicate(clean_tweet, '<USERNAME>') # Remove duplicate <USERNAME>
    clean_tweet, possesive_username_count = remove_duplicate(clean_tweet, "<USERNAME>'s") # Remove duplicate <USERNAME>'s
    clean_tweet, hashtag_count = remove_duplicate(clean_tweet, '<HASHTAG>') # Remove duplicate <HASHTAG>

    clean_tweet = replace_punct(clean_tweet, '¡', '!') # Convert upside-down exclamation points to exclamation points
    clean_tweet = replace_punct(clean_tweet, '¿', '?') # Convert upside-down question marks to question marks
    clean_tweet = re.sub(r'&', 'and', clean_tweet) # Convert ampersand to the word 'and'
    clean_tweet = re.sub(r'à', 'á', clean_tweet) # Convert à to á
    clean_tweet = re.sub(r'ª', 'a', clean_tweet) # Convert ª to a
    clean_tweet = re.sub(r'[êė]', 'e', clean_tweet) # Convert ê to e
    clean_tweet = re.sub(r'ò', 'ó', clean_tweet) # Convert ò to ó
    clean_tweet = re.sub(r'ô', 'o', clean_tweet) # Convert ô to o

    clean_tweet, exclamation_count = remove_duplicate(clean_tweet, '!') # Remove duplicate exclamation points
    clean_tweet, question_count = remove_duplicate(clean_tweet, '?') # Remove duplicate exclamation points
    clean_tweet = re.sub(r'[\u0600-\u06FF]', '', clean_tweet) # Remove Arabic characters
    clean_tweet = re.sub(r'[\u10A0-\u10FF]+', '', clean_tweet) # Remove Gregorian characters
    clean_tweet = re.sub(r'[\u4E00-\u9FFF]+', '', clean_tweet) # Remove CJK characters
    clean_tweet = re.sub(r'[\uAC00-\uD7AF]+', '', clean_tweet) # Remove Hangul characters
    clean_tweet = re.sub(r'[\u3040-\u309F]+', '', clean_tweet) # Remove hiragana
    clean_tweet = re.sub(r" '()", " ", clean_tweet) # Remove separated apostrophes
    clean_tweet = re.sub(r' +', ' ', clean_tweet) # Remove double-spaces

    return clean_tweet, username_count, exclamation_count, question_count, possesive_username_count, hashtag_count, hashtag_lst

In [56]:
# Get the DIRTY training data
eng_X_train = [value['tweet'] for key, value in training_data.items() if value['lang'] == 'en']
eng_Y_train = [1 if value['labels_task1'].count('YES') >= 3 else 0 
               for key, value in training_data.items() if value['lang'] == 'en']

# Get the DIRTY test data
eng_X_test = [value['tweet'] for key, value in dev_data.items() if value['lang'] == 'en']
eng_Y_test = [1 if value['labels_task1'].count('YES') >= 3 else 0 
               for key, value in dev_data.items() if value['lang'] == 'en']

# Sanity Check
print(len(eng_X_train) == len(eng_Y_train))
print(len(eng_X_test) == len(eng_Y_test))

True
True


In [57]:
# Clean the Data
eng_X_train_clean = [cleanTweet(tweet)[0] for tweet in eng_X_train]
eng_X_test_clean = [cleanTweet(tweet)[0] for tweet in eng_X_test]

# Sanity Check
print(len(eng_X_train_clean) == len(eng_Y_train))
print(len(eng_X_test_clean) == len(eng_Y_test))

True
True


In [58]:
# Check out the data
for dirty, clean in zip(eng_X_train[:5], eng_X_train_clean[:5]):
    print('Dirty:', dirty)
    print('Clean:', clean)
    print('-----------------------')

Dirty: FFS! How about laying the blame on the bastard who murdered her? Novel idea, I know. https://t.co/GI5B45THvJ
Clean: ffs ! how about laying the blame on the bastard who murdered her ? novel idea i know
-----------------------
Dirty: Writing a uni essay in my local pub with a coffee. Random old man keeps asking me drunk questions when I'm trying to concentrate &amp; ends with "good luck, but you'll just end up getting married and not use it anyway". #EverydaySexism is alive and well 🙃
Clean: writing a uni essay in my local pub with a coffee random old man keeps asking me drunk questions when i'm trying to concentrate and ends with good luck but you'll just end up getting married and not use it anyway <HASHTAG> is alive and well 🙃
-----------------------
Dirty: @UniversalORL it is 2021 not 1921. I dont appreciate that on two rides by myself your team member looked behind me and asked the man behind how many in my party. Not impressed #everydaysexism
Clean: <USERNAME> it is not i do

Vectorize the English Data

In [59]:
# DIRTY
# Initialize a TF-IDF Vectorizer
tfidf = TfidfVectorizer(analyzer='char', ngram_range=(2, 3))

# Fit the vectorizer on the dirty training data
tfidf.fit(eng_X_train)

# Transform the training and testing data using the vectorizer
X_train_tfidf = tfidf.transform(eng_X_train)
X_test_tfidf = tfidf.transform(eng_X_test)


print('Dirty training data shape', X_train_tfidf.shape)
print('Dirty testing data shape', X_test_tfidf.shape)



Dirty training data shape (3260, 31729)
Dirty testing data shape (489, 31729)


In [60]:
# CLEAN
# Initialize a TF-IDF Vectorizer
clean_Tfidf = TfidfVectorizer(analyzer='char', ngram_range=(2, 3))

# Fit the vectorizer on the clean training data
clean_Tfidf.fit(eng_X_train_clean)

# Transform the training and testing data using the vectorizer
X_train_clean_tfidf = tfidf.transform(eng_X_train_clean)
X_test_clean_tfidf = tfidf.transform(eng_X_test_clean)


print('Clean training data shape:', X_train_clean_tfidf.shape)
print('Clean testing data shape:', X_test_clean_tfidf.shape)


Clean training data shape: (3260, 31729)
Clean testing data shape: (489, 31729)


Train the English Models

In [61]:
# DIRTY MODEL
# Perform hyperparameter optimization and train the models

svm = SVC()


# Define the hyperparameters to tune
params = {
    'C': [0.1, 1, 10],
    'gamma': [0.1, 1, 10],
    'kernel': ['linear', 'poly', 'rbf']
}

# Perform a grid search over the hyperparameters
svm_gs = GridSearchCV(svm, params)
svm_gs.fit(X_train_tfidf, eng_Y_train)

# Print the best hyperparameters found
print("Best parameters:", svm_gs.best_params_)

# Predict the labels of the test data using the trained classifier with the best hyperparameters
y_pred = svm_gs.predict(X_test_tfidf)

Best parameters: {'C': 1, 'gamma': 1, 'kernel': 'rbf'}


In [62]:
# Print the classification report and accuracy score with the best hyperparameters
print(classification_report(eng_Y_test, y_pred, digits=4))
print("Accuracy:", accuracy_score(eng_Y_test, y_pred))

              precision    recall  f1-score   support

           0     0.6964    0.8440    0.7631       250
           1     0.7903    0.6151    0.6918       239

    accuracy                         0.7321       489
   macro avg     0.7433    0.7295    0.7274       489
weighted avg     0.7423    0.7321    0.7282       489

Accuracy: 0.7321063394683026


In [63]:
# CLEAN MODEL
# Perform hyperparameter optimization and train the models

clean_svm = SVC()


# Define the hyperparameters to tune
params = {
    'C': [0.1, 1, 10],
    'gamma': [0.1, 1, 10],
    'kernel': ['linear', 'poly', 'rbf']
}

# Perform a grid search over the hyperparameters
clean_svm_gs = GridSearchCV(clean_svm, params)
clean_svm_gs.fit(X_train_clean_tfidf, eng_Y_train)

# Print the best hyperparameters found
print("Best parameters:", clean_svm_gs.best_params_)

# Predict the labels of the test data using the trained classifier with the best hyperparameters
clean_y_pred = clean_svm_gs.predict(X_test_clean_tfidf)

# Print the classification report and accuracy score with the best hyperparameters
print(classification_report(eng_Y_test, clean_y_pred, digits=4))
print("Accuracy:", accuracy_score(eng_Y_test, clean_y_pred))

Best parameters: {'C': 1, 'gamma': 0.1, 'kernel': 'rbf'}
              precision    recall  f1-score   support

           0     0.6528    0.8800    0.7496       250
           1     0.8026    0.5105    0.6240       239

    accuracy                         0.6994       489
   macro avg     0.7277    0.6952    0.6868       489
weighted avg     0.7260    0.6994    0.6882       489

Accuracy: 0.6993865030674846
